In [1]:
# import necessary library
import pandas as pd

In [2]:
# fetch the file using a link; unix core
!curl -O https://ftp.ncbi.nih.gov/genomes/refseq/vertebrate_mammalian/Homo_sapiens/latest_assembly_versions/GCF_000001405.39_GRCh38.p13/GCF_000001405.39_GRCh38.p13_genomic.fna.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  920M  100  920M    0     0   188M      0  0:00:04  0:00:04 --:--:--  200M


In [3]:
!gunzip /content/GCF_000001405.39_GRCh38.p13_genomic.fna.gz

In [4]:
# install the pyfaidx 
!pip install pyfaidx

  Created wheel for pyfaidx: filename=pyfaidx-0.5.9.5-cp37-none-any.whl size=25142 sha256=5246b94d086fdc1df3a39903681b16ee53fdb02ad17e200f7b3271f5792cf584
  Stored in directory: /root/.cache/pip/wheels/d8/31/5f/8053c426a420cc407492252723f20e9a9c7e717909d7e08a9e
Successfully built pyfaidx


In [5]:
# import the pyfaidx
from pyfaidx import Fasta

In [6]:
# read all fasta chromosoms and convert it to upper letters
genes = Fasta('/content/GCF_000001405.39_GRCh38.p13_genomic.fna', sequence_always_upper=True)

In [7]:
# import the key of genes 
Fasta_Keys = genes.keys()

In [8]:
# extract the fasta keys from the human fasta file 
df_Fasta_Keys = pd.DataFrame(Fasta_Keys,columns=['Fasta_Keys'])

In [10]:
# Read the file of cluster 1
DF_class1 = pd.read_csv('/content/ensemble_genes_IDs_updated_df3.csv')
DF_class1.head()

,chrom,chromStart,chromEnd,strand,Gene_ID
0,NC_000001.11,944203,959256,-,ENSG00000188976
1,NC_000001.11,1013497,1014540,+,ENSG00000187608
2,NC_000001.11,1020102,1056119,+,ENSG00000188157
3,NC_000001.11,1216908,1232005,-,ENSG00000078808
4,NC_000001.11,1232237,1235041,+,ENSG00000176022


In [11]:
# chromosome code and a subsequence unclutide from 944203 to 959256 
genes['NC_000001.11'][944203:959256]

>NC_000001.11:944204-959256
CAACACAATGGCCCTGCCTCCCACCGCTTTATTTCTTTCGGTTTCGGATGCAAAACAAAAAATTTTAAAAGAAAATGTGACTTCAAAGGAAAGGAACAAATTTTCAAAGACTTGGGGGAGTGAAGGCAGAGCCTGGTGCAGATGGACGAGGTCTGCAGACGGAGGGCAGAGGTGGTGGAAGGGGCCAGGGGCCTGCAGGCCTCCCCCTGGAACTGGGACTGGTCTCGGTCTGCTGACGTCAGGGTCAGCTCCCCCGCGGAGCTGACTTCAGCAGCCCACAGCTGTGGGGCTTCAGCAGCCACACCAGCCCAGCCCAGCCCAGCTCTCGATACGTTTGGTCTTTCATGCTGAAAAATAAATAATAAAGCCTGTCCCGTGTCTACTGCCTCCCCCAACTGCACAGACGCCAGCCTCTAGGCCTGACTGCCAGGGAGGTGGAAACACTGGCCACCAGCCCGGCAGCCCCTACAGGCCCCCCAGATGGGCTGCCTCAGTCGTCCTCTGAGAGCTGCAGATCCTCCAGCTCGTCCTCCGGCCCCTGGGCCAGCTGCTGCAGCTCCCCAGGGGCCAGCCCCGCCTCTGCGTCTGGGTCTCCATCTGCGGGGAGAGATGGAGGCTACATAAATTTTGCTTTATCAGGAAGAAGCCAGCCTTAGAGGTTACTCATCACTAATTAATCACGGCACTAATTAATTTATCCCTGTTGCTGGCTGCCAGAGAACAGAGCATTTGGCCTGGCCTTCCCAGGGAGGGAAAAGCCTGGCCCAGAGCCCCACGCCCCCCGCCCACGTGGCTCTGCCCTCCCGCCAGATGGGCTCACAGGGCCACACCCTCTCACCCCAAGACCATTCACCCTCCGAGTTGCTGCTGTCCTCCTCGCCCTCCTCCTCGTCCTCTTCATCGTCTTCCACCCCATGCCGAGTGCTCAGGGGCCTCAGTATCCCTGAGGAACAAGAAGCAGAGTCCATATGA

In [12]:
# Chromosome 500 in chrome column
genes[DF_class1.iloc[500]['chrom ']]

FastaRecord("NC_000001.11")

In [16]:
!pip install realpython-reader

     |████████████████████████████████| 81kB 3.5MB/s 
     |████████████████████████████████| 81kB 3.5MB/s 
  Created wheel for typing: filename=typing-3.7.4.3-cp37-none-any.whl size=26308 sha256=583f55c4b878c9e3775b95b19183fc896813d9228da09bbc9630a717388ff490
  Stored in directory: /root/.cache/pip/wheels/2d/04/41/8e1836e79581989c22eebac3f4e70aaac9af07b0908da173be
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-cp37-none-any.whl size=6067 sha256=d856eb7b207428e1c3882cb77b413995bb3c6a87e3dafc87c816528402a8f60d
  Stored in directory: /root/.cache/pip/wheels/f1/80/5a/444ba08a550cdd241bd9baf8bae44be750efe370adb944506a
Successfully built typing sgmllib3k


In [17]:
# Fetch the sequence based on the start and the end of genes, 
#then replace the gene ensembl with the chromosome
tic = time.perf_counter()
# for saving the result of sequence and ensembl gene id 
new_fasta = []
for gen in range(len(df_Fasta_Keys)):
  #starting nuectuotide
  start = DF_class1.iloc[gen]['chromStart']
  #ending nuectuotide
  end = DF_class1.iloc[gen]['chromEnd']
  #apply a function based on the strand
  #inverse and complement
  if DF_class1.iloc[gen]['strand'] == '-':
    seq = -genes[DF_class1.iloc[gen]['chrom ']][start:end]
    new_fasta.append('>%s\n%s' % (DF_class1.iloc[gen]['Gene_ID'], seq))
  else:
    # same subsequence
    seq = genes[DF_class1.iloc[gen]['chrom ']][start:end]
    new_fasta.append('>%s\n%s' % (DF_class1.iloc[gen]['Gene_ID'], seq))
toc = time.perf_counter()
print(f"The consumed time in the background genes file is {toc - tic:0.4f} seconds")

The consumed time in the background genes file is 0.7156 seconds


In [14]:
# saving the result of mapped ensembl with the selected portion to a fasta file
with open('genes_DF_Background.fasta', 'w') as f:
    f.write('\n'.join(new_fasta))